In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/national-health-and-nutrition-examination-survey/examination.csv
/kaggle/input/national-health-and-nutrition-examination-survey/medications.csv
/kaggle/input/national-health-and-nutrition-examination-survey/demographic.csv
/kaggle/input/national-health-and-nutrition-examination-survey/labs.csv
/kaggle/input/national-health-and-nutrition-examination-survey/questionnaire.csv
/kaggle/input/national-health-and-nutrition-examination-survey/diet.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

demo = pd.read_csv('/kaggle/input/national-health-and-nutrition-examination-survey/demographic.csv',
                   usecols=['SEQN', 'RIAGENDR', 'RIDAGEYR', 'DMDEDUC2', 'INDFMPIR'])

exam = pd.read_csv('/kaggle/input/national-health-and-nutrition-examination-survey/examination.csv',
                   usecols=['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1'])

labs = pd.read_csv('/kaggle/input/national-health-and-nutrition-examination-survey/labs.csv',
                   usecols=['SEQN', 'LBXSGL', 'LBDSGLSI', 'LBXTC', 'LBDTCSI', 'LBDHDD', 'LBDHDDSI'])

quest = pd.read_csv('/kaggle/input/national-health-and-nutrition-examination-survey/questionnaire.csv',
                    usecols=['SEQN', 'SMQ020', 'ALQ130', 'PAQ605', 'PAQ620', 'MCQ160F'])

df = demo.merge(exam, on='SEQN') \
         .merge(labs, on='SEQN') \
         .merge(quest, on='SEQN')

df = df.dropna()
print("Final shape after merging and dropping NAs:", df.shape)

def glucose_status(glu):
    if glu <= 100:
        return 0
    elif 100 < glu <= 125:
        return 1
    else:
        return 2

df['target'] = df['LBXSGL'].apply(glucose_status)

X = df.drop(columns=['SEQN', 'target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = XGBClassifier(objective='multi:softmax', num_class=3, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=["Normal", "Pre-diabetic", "Diabetic"]))


Final shape after merging and dropping NAs: (2865, 19)
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       384
Pre-diabetic       1.00      1.00      1.00       131
    Diabetic       1.00      1.00      1.00        58

    accuracy                           1.00       573
   macro avg       1.00      1.00      1.00       573
weighted avg       1.00      1.00      1.00       573

